<a href="https://colab.research.google.com/github/twyeh/AI-in-education/blob/main/SNN_for_inclined_projectile_motion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input # Import Input layer
from tensorflow.keras.optimizers import Adam

# 產生斜拋運動的訓練資料
def generate_data(num_samples=10000):
    g = 9.8  # 重力加速度
    v0 = np.random.uniform(10, 50, num_samples)           # 初速度 (10~50 m/s)
    theta = np.random.uniform(0, np.pi/2, num_samples)    # 拋射角度 (0~90度, 轉弧度)
    t = np.random.uniform(0, 10, num_samples)             # 時間 (0~10秒)

    x = v0 * np.cos(theta) * t
    y = v0 * np.sin(theta) * t - 0.5 * g * t**2

    # 輸入為 time, v0, theta 三個變數
    X = np.stack([t, v0, theta], axis=1)
    # 輸出為 x, y 兩個位置值
    Y = np.stack([x, y], axis=1)

    return X, Y

# 建立神經網路模型
def build_model():
    model = Sequential([
        Input(shape=(3,)),
        Dense(64, activation='relu'),
        Dense(64, activation='relu'),
        Dense(2)  # 輸出x和y坐標
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return model

# 產生資料
X_train, Y_train = generate_data(20000)

# 創建模型
model = build_model()

# 訓練模型
model.fit(X_train, Y_train, epochs=20, batch_size=64, validation_split=0.2)

# 測試：輸入時間為2秒，初速度50m/s，角度45度(弧度0.785)
test_input = np.array([[2.0, 50.0, np.pi/4]])
predicted_position = model.predict(test_input)
print(f"預測位置 (x, y): {predicted_position[0]}")

Epoch 1/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 13121.7490 - val_loss: 5122.1592
Epoch 2/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 4101.0425 - val_loss: 2766.4590
Epoch 3/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 2566.2952 - val_loss: 1912.4510
Epoch 4/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1723.9648 - val_loss: 1328.9990
Epoch 5/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1239.6406 - val_loss: 1034.0287
Epoch 6/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 985.1047 - val_loss: 882.7141
Epoch 7/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 863.3345 - val_loss: 783.1308
Epoch 8/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 737.5663 - val_loss: 687.7902
Epoch 9/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 661.5458 - val_loss: 611.9919
Epoch 10/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 613.3434 - val_loss: 560.1088
Epoch 11/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 542.9087 - val_loss: 516.16

In [6]:
from sklearn.metrics import mean_absolute_error

# Evaluate on training and validation data using MAE
train_preds = model.predict(X_train)
mae_train = mean_absolute_error(Y_train, train_preds)
print(f"Mean Absolute Error on training data: {mae_train:.4f}")

# Assuming validation data is available from the training split
# We can split the training data again to get validation data for evaluation
split_index = int(len(X_train) * 0.8)
X_val, Y_val = X_train[split_index:], Y_train[split_index:]

val_preds = model.predict(X_val)
mae_val = mean_absolute_error(Y_val, val_preds)
print(f"Mean Absolute Error on validation data: {mae_val:.4f}")

# Test with a few known examples
test_cases = [
    [2.0, 50.0, np.pi/4],  # t=2, v0=50, theta=45 deg
    [3.0, 30.0, np.pi/6],  # t=3, v0=30, theta=30 deg
    [1.5, 40.0, np.pi/3]   # t=1.5, v0=40, theta=60 deg
]

print("\nTesting with known examples:")
for t, v0, theta in test_cases:
    # Calculate true position
    x_true = v0 * np.cos(theta) * t
    y_true = v0 * np.sin(theta) * t - 0.5 * 9.8 * t**2

    # Predict with the model
    test_input = np.array([[t, v0, theta]])
    predicted_position = model.predict(test_input, verbose=0)[0]

    print(f"Input: t={t}, v0={v0}, theta={np.degrees(theta):.2f} deg")
    print(f"True position (x, y): ({x_true:.2f}, {y_true:.2f})")
    print(f"Predicted position (x, y): ({predicted_position[0]:.2f}, {predicted_position[1]:.2f})")
    print("-" * 20)

625/625 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
Mean Absolute Error on training data: 13.3458
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
Mean Absolute Error on validation data: 13.4419

Testing with known examples:
Input: t=2.0, v0=50.0, theta=45.00 deg
True position (x, y): (70.71, 51.11)
Predicted position (x, y): (57.61, 36.55)
--------------------
Input: t=3.0, v0=30.0, theta=30.00 deg
True position (x, y): (77.94, 0.90)
Predicted position (x, y): (66.37, 20.29)
--------------------
Input: t=1.5, v0=40.0, theta=60.00 deg
True position (x, y): (30.00, 40.94)
Predicted position (x, y): (23.13, 38.98)
--------------------


# Task
Tune the hyperparameters of the model to improve its performance.

## Define hyperparameter search space

### Subtask:
Specify the range of values or options to explore for hyperparameters like learning rate, number of layers, neurons per layer, and activation functions.


**Reasoning**:
Define the hyperparameter search space as a dictionary.



In [7]:
# Define the hyperparameter search space
param_grid = {
    'learning_rate': [0.01, 0.001, 0.0001],
    'num_layers': [1, 2, 3],
    'neurons_per_layer': {
        1: [32, 64, 128],
        2: [(32, 32), (64, 32), (64, 64), (128, 64)],
        3: [(32, 32, 32), (64, 32, 16), (128, 64, 32)]
    },
    'activation': ['relu', 'tanh']
}

print(param_grid)

{'learning_rate': [0.01, 0.001, 0.0001], 'num_layers': [1, 2, 3], 'neurons_per_layer': {1: [32, 64, 128], 2: [(32, 32), (64, 32), (64, 64), (128, 64)], 3: [(32, 32, 32), (64, 32, 16), (128, 64, 32)]}, 'activation': ['relu', 'tanh']}


## Implement hyperparameter tuning

### Subtask:
Use a library or custom code to systematically train and evaluate the model with different hyperparameter combinations. This could involve techniques like grid search, random search, or more advanced methods like Bayesian optimization.


**Reasoning**:
Define a function to build the model with specified hyperparameters, generate and split the data, and iterate through hyperparameter combinations to train and evaluate models, keeping track of the best one.



In [8]:
def build_tunable_model(num_layers, neurons_per_layer, activation, learning_rate):
    """Builds a Keras Sequential model with specified hyperparameters."""
    model = Sequential()
    model.add(Input(shape=(3,))) # Input layer

    if num_layers == 1:
        model.add(Dense(neurons_per_layer, activation=activation))
    elif num_layers > 1:
        for i, neurons in enumerate(neurons_per_layer):
            model.add(Dense(neurons, activation=activation))

    model.add(Dense(2))  # Output layer for x, y

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])
    return model

# Generate data
X, Y = generate_data(20000)

# Split data
split_ratio = 0.8
split_index = int(len(X) * split_ratio)
X_train, X_val = X[:split_index], X[split_index:]
Y_train, Y_val = Y[:split_index], Y[split_index:]

best_mae = float('inf')
best_params = None
history_list = []

# Iterate through hyperparameter combinations
for lr in param_grid['learning_rate']:
    for num_layers in param_grid['num_layers']:
        neurons_options = param_grid['neurons_per_layer'][num_layers]
        if num_layers == 1:
             # neurons_options is a list of integers for num_layers == 1
             for neurons in neurons_options:
                for activation in param_grid['activation']:
                    print(f"Training with: LR={lr}, Layers={num_layers}, Neurons={neurons}, Activation={activation}")
                    model = build_tunable_model(num_layers, neurons, activation, lr)
                    history = model.fit(X_train, Y_train, epochs=20, batch_size=64, validation_data=(X_val, Y_val), verbose=0)
                    val_mae = history.history['val_mae'][-1]
                    print(f"Validation MAE: {val_mae:.4f}")

                    if val_mae < best_mae:
                        best_mae = val_mae
                        best_params = {'learning_rate': lr, 'num_layers': num_layers, 'neurons': neurons, 'activation': activation}

                    history_list.append({'params': {'learning_rate': lr, 'num_layers': num_layers, 'neurons': neurons, 'activation': activation},
                                         'history': history.history})
        else:
            # neurons_options is a list of tuples for num_layers > 1
            for neurons in neurons_options:
                for activation in param_grid['activation']:
                    print(f"Training with: LR={lr}, Layers={num_layers}, Neurons={neurons}, Activation={activation}")
                    model = build_tunable_model(num_layers, neurons, activation, lr)
                    history = model.fit(X_train, Y_train, epochs=20, batch_size=64, validation_data=(X_val, Y_val), verbose=0)
                    val_mae = history.history['val_mae'][-1]
                    print(f"Validation MAE: {val_mae:.4f}")

                    if val_mae < best_mae:
                        best_mae = val_mae
                        best_params = {'learning_rate': lr, 'num_layers': num_layers, 'neurons': neurons, 'activation': activation}

                    history_list.append({'params': {'learning_rate': lr, 'num_layers': num_layers, 'neurons': neurons, 'activation': activation},
                                         'history': history.history})


print("\nBest hyperparameters found:")
print(best_params)
print(f"Best Validation MAE: {best_mae:.4f}")

Training with: LR=0.01, Layers=1, Neurons=32, Activation=relu
Validation MAE: 10.2303
Training with: LR=0.01, Layers=1, Neurons=32, Activation=tanh
Validation MAE: 10.8349
Training with: LR=0.01, Layers=1, Neurons=64, Activation=relu
Validation MAE: 8.7818
Training with: LR=0.01, Layers=1, Neurons=64, Activation=tanh
Validation MAE: 6.1706
Training with: LR=0.01, Layers=1, Neurons=128, Activation=relu
Validation MAE: 5.9379
Training with: LR=0.01, Layers=1, Neurons=128, Activation=tanh
Validation MAE: 3.5610
Training with: LR=0.01, Layers=2, Neurons=(32, 32), Activation=relu
Validation MAE: 4.1487
Training with: LR=0.01, Layers=2, Neurons=(32, 32), Activation=tanh
Validation MAE: 3.3994
Training with: LR=0.01, Layers=2, Neurons=(64, 32), Activation=relu
Validation MAE: 4.1484
Training with: LR=0.01, Layers=2, Neurons=(64, 32), Activation=tanh
Validation MAE: 5.6846
Training with: LR=0.01, Layers=2, Neurons=(64, 64), Activation=relu
Validation MAE: 3.1737
Training with: LR=0.01, Layers=

## Retrain the best model

### Subtask:
Train the model with the selected hyperparameters on the entire training dataset.


**Reasoning**:
Define a new model using the `build_tunable_model` function with the `best_params` and train it on the entire dataset.



In [9]:
# Define a new model with the best hyperparameters
tuned_model = build_tunable_model(
    num_layers=best_params['num_layers'],
    neurons_per_layer=best_params['neurons'],
    activation=best_params['activation'],
    learning_rate=best_params['learning_rate']
)

# Train the model on the entire dataset (X and Y)
print("Training the final model with best hyperparameters on the full dataset...")
tuned_model.fit(X, Y, epochs=20, batch_size=64)


Training the final model with best hyperparameters on the full dataset...
Epoch 1/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 4746.7920 - mae: 44.9490
Epoch 2/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 434.1216 - mae: 15.7311
Epoch 3/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 373.1495 - mae: 14.4809
Epoch 4/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 224.4123 - mae: 11.0847
Epoch 5/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 86.7375 - mae: 6.7907
Epoch 6/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 55.8162 - mae: 5.3030
Epoch 7/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 29.9636 - mae: 3.9880
Epoch 8/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 28.9228 - mae: 3.9097
Epoch 9/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 38.4787 - mae: 4.3234
Epoch 10/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 27.9754 - mae: 3.7702
Epoch 11/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 17.7233 - mae: 3.1013
Epoch 12/20
3

## Evaluate the final model

### Subtask:
Evaluate the performance of the retrained model on a separate test set (if available) or using cross-validation to get a more robust estimate of its performance.


**Reasoning**:
Generate a separate test dataset and evaluate the performance of the retrained model on it using MAE.



In [10]:
# Generate a new, separate test dataset
X_test, Y_test = generate_data(5000) # Generate 5000 test samples

# Evaluate the tuned model on the test dataset
print("\nEvaluating the tuned model on the test dataset...")
test_results = tuned_model.evaluate(X_test, Y_test, verbose=0)

# Extract the MAE from the evaluation results
# Assuming 'mae' was the metric used during compilation
mae_on_test = test_results[1]

# Print the MAE on the test data
print(f"Mean Absolute Error on test data: {mae_on_test:.4f}")


Evaluating the tuned model on the test dataset...
Mean Absolute Error on test data: 3.0325


## Summary:

### Data Analysis Key Findings

*   The hyperparameter tuning process identified the best combination of hyperparameters from the defined search space as: Learning Rate: 0.01, Number of Layers: 3, Neurons per Layer: (128, 64, 32), and Activation Function: 'relu'.
*   This best combination achieved the lowest validation MAE of 2.2430 during the tuning phase.
*   After retraining the model with the best hyperparameters on the entire dataset, the evaluation on a separate test set resulted in a Mean Absolute Error of approximately 3.0325.

### Insights or Next Steps

*   The difference between the validation MAE (2.2430) and the test MAE (3.0325) suggests potential overfitting during the tuning process or training on the full dataset. Further investigation into regularization techniques or cross-validation during tuning could be beneficial.
*   Explore a wider range of hyperparameters, including different activation functions, optimizer types, and regularization parameters, to potentially achieve better generalization on unseen data.
